<a href="https://colab.research.google.com/github/krishna-kenny/nbaWinNeuralNetModel/blob/main/nba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install nba_api

In [31]:
import time
import pandas as pd
from nba_api.stats.endpoints import TeamInfoCommon, TeamGameLogs, PlayerGameLogs, LeagueGameFinder, LeagueLeaders, PlayerCareerStats
from nba_api.stats.static import teams

# Maximum number of retries for each API call
MAX_RETRIES = 3
# Define the list of seasons
# Generate all seasons from 2013 onwards
start_year = 2024
end_year = 2024  # Adjust to your desired year
seasons = [f"{year}-{(year + 1)%100}" for year in range(start_year, end_year + 1)]

# Printing seasons to verify
print(seasons)


['2024-25']


In [32]:
def fetch_with_retries(func, *args, **kwargs):
    """Attempts a function call up to MAX_RETRIES with exponential backoff."""
    for attempt in range(MAX_RETRIES):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            wait_time = 2**attempt  # Exponential backoff
            print(f"Error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    print(f"Failed after {MAX_RETRIES} attempts.")
    return None

In [33]:
def get_team_info(seasons):
    """Fetches relevant team information for the specified seasons."""
    print("Fetching team information...")
    nba_teams = teams.get_teams()
    team_data = []

    for team in nba_teams:
        team_info = fetch_with_retries(
            TeamInfoCommon,
            team_id=team["id"],
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if team_info:
            df_team = team_info.get_data_frames()[0]
            df_team = df_team[["TEAM_ID", "TEAM_ABBREVIATION"]]  # Only keep relevant features
            team_data.append(df_team)
            time.sleep(0.6)  # Delay to avoid API rate limits

    if team_data:
        df_teams = pd.concat(team_data, ignore_index=True)
        df_teams.to_csv("nba_team_data.csv", index=False)
    else:
        print("No team data fetched.")

# Run functions to save data to CSV files
get_team_info(seasons)
print("Team information data stored.")

Fetching team information...
Team information data stored.


In [34]:
def get_team_game_logs(seasons):
    """Fetches team game logs for the specified seasons and processes the MATCHUP column."""
    print("Fetching team game logs...")
    game_log_data = []

    for season in seasons:
        game_logs = fetch_with_retries(
            TeamGameLogs,
            season_nullable=season,
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if game_logs:
            df_game_logs = game_logs.get_data_frames()[0]
            # Keep only relevant columns
            df_game_logs = df_game_logs[["GAME_ID", "GAME_DATE", "MATCHUP", "WL"]]
            game_log_data.append(df_game_logs)
            time.sleep(0.6)  # Delay to respect rate limits

    if game_log_data:
        # Concatenate all game logs
        df_all_game_logs = pd.concat(game_log_data, ignore_index=True)

        # Process MATCHUP column to create team1 and team2 columns
        matchups_split = df_all_game_logs['MATCHUP'].str.split(' @ | vs. ', expand=True)
        df_all_game_logs['TEAM1'] = matchups_split[0]
        df_all_game_logs['TEAM2'] = matchups_split[1]

        # Drop the original MATCHUP column if no longer needed
        df_all_game_logs.drop(columns=['MATCHUP'], inplace=True)

        # Extract and add SEASON_YEAR
        df_all_game_logs['SEASON_YEAR'] = pd.to_datetime(df_all_game_logs['GAME_DATE']).dt.year.astype(str)

        # Create combined TEAM_SEASON columns
        df_all_game_logs['TEAM_SEASON1'] = df_all_game_logs['TEAM1'] + ':' + df_all_game_logs['SEASON_YEAR']
        df_all_game_logs['TEAM_SEASON2'] = df_all_game_logs['TEAM2'] + ':' + df_all_game_logs['SEASON_YEAR']

        # Drop the original TEAM1, TEAM2, and SEASON_YEAR columns if no longer needed
        df_all_game_logs.drop(columns=['TEAM1', 'TEAM2', 'SEASON_YEAR'], inplace=True)

        # Save the processed DataFrame to a CSV file
        df_all_game_logs.to_csv("nba_game_logs.csv", index=False)
        print("Processed game logs saved to 'nba_game_logs.csv'.")
    else:
        print("No game log data fetched.")

get_team_game_logs(seasons)
print("Team game logs data stored.")


Fetching team game logs...
Processed game logs saved to 'nba_game_logs.csv'.
Team game logs data stored.


In [35]:
def get_player_game_logs(seasons):
    """Fetches player game logs for the specified seasons."""
    print("Fetching player game logs...")
    player_game_log_data = []

    for season in seasons:
        player_game_logs = fetch_with_retries(
            PlayerGameLogs,
            season_nullable=season,
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if player_game_logs:
            df_player_game_logs = player_game_logs.get_data_frames()[0]
            # Keep only relevant columns
            df_player_game_logs = df_player_game_logs[[
                "SEASON_YEAR", "GAME_ID", "TEAM_ID", "PLAYER_ID", "PLAYER_NAME", "PTS", "REB", "AST", "STL", "BLK",
                "MIN", "FG_PCT", "FG3_PCT", "FT_PCT", "TOV", "PF"
            ]]
            # Modify SEASON_YEAR to keep only the first 4 characters
            df_player_game_logs["SEASON_YEAR"] = df_player_game_logs["SEASON_YEAR"].str[:4]
            player_game_log_data.append(df_player_game_logs)
            time.sleep(0.6)

    if player_game_log_data:
        df_all_player_game_logs = pd.concat(player_game_log_data, ignore_index=True)
        df_all_player_game_logs.to_csv("nba_player_game_logs.csv", index=False)
    else:
        print("No player game log data fetched.")

# Call the function with the specified seasons
get_player_game_logs(seasons)
print("Player game logs data stored.")

Fetching player game logs...
Player game logs data stored.


In [36]:
def get_league_game_data():
    """Fetches league-wide game data with relevant features for a neural network."""
    print("Fetching league game data for NN...")
    game_data = fetch_with_retries(LeagueGameFinder, timeout=60)
    if game_data:
        df_game_data = game_data.get_data_frames()[0]
        # Relevant columns for neural network input
        relevant_columns = [
            "SEASON_ID", "TEAM_ID", "TEAM_ABBREVIATION", "TEAM_NAME", "GAME_ID",
            "GAME_DATE", "MATCHUP", "WL", "MIN", "PTS", "FGM", "FGA", "FG_PCT",
            "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB", "DREB",
            "REB", "AST", "STL", "BLK", "TOV", "PF", "PLUS_MINUS"
        ]
        df_nn_data = df_game_data[relevant_columns]
        df_nn_data.to_csv("nba_league_game.csv", index=False)
    else:
        print("No league game data fetched.")

get_league_game_data()
print("League game data stored.")

Fetching league game data for NN...
League game data stored.


In [37]:
def get_league_leaders():
    """Fetches league leaders data with relevant columns for analysis."""
    print("Fetching league leaders data...")
    leaders_data = fetch_with_retries(LeagueLeaders, timeout=60)
    if leaders_data:
        df_leaders = leaders_data.get_data_frames()[0]
        # Select only relevant columns
        relevant_columns = [
            "PLAYER_ID", "PLAYER", "TEAM_ID", "TEAM", "GP", "MIN", "FGM", "FGA",
            "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB",
            "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "EFF"
        ]
        df_relevant_leaders = df_leaders[relevant_columns]
        df_relevant_leaders.to_csv("nba_league_leaders_relevant.csv", index=False)
    else:
        print("No league leaders data fetched.")

get_league_leaders()
print("League leaders data stored.")

Fetching league leaders data...
League leaders data stored.


In [38]:
def get_player_career_stats():
    """Fetches career stats for players."""
    print("Fetching player career stats...")
    career_stats_data = []
    nba_teams = teams.get_teams()
    for team in nba_teams:
        players = team.get("players", [])
        for player in players:
            career_stats = fetch_with_retries(PlayerCareerStats, player_id=player["id"], timeout=60)
            if career_stats:
                df_career_stats = career_stats.get_data_frames()[0]
                # Keep only relevant columns
                df_career_stats = df_career_stats[[
                    "PLAYER_ID", "PLAYER_NAME", "GP", "PTS", "REB", "AST", "FG_PCT", "FG3_PCT", "FT_PCT"
                ]]
                career_stats_data.append(df_career_stats)
                time.sleep(0.6)

    if career_stats_data:
        df_all_career_stats = pd.concat(career_stats_data, ignore_index=True)
        df_all_career_stats.to_csv("nba_player_career_stats.csv", index=False)
    else:
        print("No player career stats data fetched.")

get_player_career_stats()
print("Player career stats data stored.")

Fetching player career stats...
No player career stats data fetched.
Player career stats data stored.


In [39]:

import pandas as pd

# Load player game logs from the CSV file
file_path = "nba_player_game_logs.csv"  # Update with your actual file path
df = pd.read_csv(file_path)

# Exclude non-numerical columns explicitly
no_aggregate_columns = ['PLAYER_ID', 'SEASON_YEAR', 'PLAYER_NAME', 'TEAM_ID']  # Adjust as necessary
numerical_columns = [col for col in df.columns if col not in no_aggregate_columns]

# Group by PLAYER_ID and SEASON_YEAR
grouped = df.groupby(['PLAYER_ID', 'SEASON_YEAR'])

# Aggregate numerical columns with mean, std, median, and variance
aggregated_data = grouped[numerical_columns].agg(['mean', 'std', 'median', 'var']).reset_index()

# Flatten multi-level columns
aggregated_data.columns = ['_'.join(col).strip('_') for col in aggregated_data.columns]

# Add coefficient of variation (CV) separately
for col in numerical_columns:
    col_mean = f"{col}_mean"
    col_std = f"{col}_std"
    col_cv = f"{col}_cv"
    aggregated_data[col_cv] = aggregated_data[col_std] / aggregated_data[col_mean]
    aggregated_data[col_cv] = aggregated_data[col_cv].replace([float('inf'), -float('inf')], None)  # Handle division by zero

# Add non-numerical columns using the first value in the group (like TEAM_ID)
aggregated_data['TEAM_ID'] = grouped['TEAM_ID'].first().values

# Add games played as a new column
aggregated_data['GAMES_PLAYED'] = grouped.size().values

# Save the aggregated data for further use
output_path = "nba_player_aggregated_data.csv"
aggregated_data.to_csv(output_path, index=False)

print(f"Aggregated data saved to '{output_path}'.")

Aggregated data saved to 'nba_player_aggregated_data.csv'.


In [41]:

import pandas as pd

# Load aggregated player data
player_aggregated_file = "nba_player_aggregated_data.csv"  # Update with your actual file path
team_abbreviation_file = "nba_team_data.csv"  # File containing TEAM_ID to TEAM_ABBREVIATION mapping

# Load player data and team abbreviation mapping
player_df = pd.read_csv(player_aggregated_file)
team_data_df = pd.read_csv(team_abbreviation_file)

# Ensure 'MIN' column exists
if 'MIN' not in player_df.columns:
    raise KeyError("The 'MIN' column is missing from the player data. Please verify the input file.")

# Merge team abbreviations into player data
player_df = player_df.merge(team_data_df[['TEAM_ID', 'TEAM_ABBREVIATION']], on="TEAM_ID", how="left")

# Combine TEAM_ABBREVIATION and SEASON_YEAR into a new column
player_df['TEAM_SEASON'] = player_df['TEAM_ABBREVIATION'] + ":" + player_df['SEASON_YEAR'].astype(str)

# Drop unnecessary columns
columns_to_drop = ['TEAM_ABBREVIATION', 'SEASON_YEAR']
if 'GAME_ID' in player_df.columns:  # Check if 'GAME_ID' exists
    columns_to_drop.append('GAME_ID')

player_df.drop(columns=columns_to_drop, inplace=True)

# Define non-numerical columns to exclude
no_aggregate_columns = ['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_SEASON']
numerical_columns = [col for col in player_df.columns if col not in no_aggregate_columns]

# Ensure previous aggregate features (mean, median, std, cv) are incorporated
aggregate_stats = ['mean', 'median', 'std', 'cv']
numerical_columns = [
    col for col in numerical_columns
    if any(col.endswith(stat) for stat in aggregate_stats)
]

# Multiply each player's stats by their 'MIN' to weight the statistics
for col in numerical_columns:
    player_df[f"{col}_WEIGHTED"] = player_df[col] * player_df['MIN']

# Group by TEAM_SEASON
grouped = player_df.groupby(['TEAM_SEASON'])

# Compute team-level weighted stats as the sum of weighted stats
weighted_stats = grouped[[f"{col}_WEIGHTED" for col in numerical_columns]].sum()

# Rename back to original column names by stripping '_WEIGHTED'
weighted_stats.columns = [col.replace('_WEIGHTED', '') for col in weighted_stats.columns]

# Compute total minutes played by the team
total_minutes = grouped['MIN'].sum()

# Normalize weighted stats by dividing by total minutes
team_aggregated_data = weighted_stats.div(total_minutes, axis=0)
team_aggregated_data['TOTAL_MIN'] = total_minutes

# Add additional columns
team_aggregated_data['TEAM_GAMES_PLAYED'] = grouped['GAMES_PLAYED'].sum()

# Reset index to flatten the DataFrame
team_aggregated_data.reset_index(inplace=True)

# Save the aggregated data for further use
output_path = "nba_team_aggregated_data.csv"
team_aggregated_data.to_csv(output_path, index=False)

print(f"Team aggregated data saved to '{output_path}'.")
print(team_aggregated_data.head())

KeyError: "The 'MIN' column is missing from the player data. Please verify the input file."

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [ ]:
def extract_season_year(game_date):
    """
    Extracts the year from a game date in the format 'YYYY-MM-DDTHH:MM:SS'.

    Args:
        game_date (str): The game date string.

    Returns:
        str: The year as a string.
    """
    try:
        return pd.to_datetime(game_date).year
    except Exception as e:
        print(f"Error parsing date '{game_date}': {e}")
        return None

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
import joblib
import tensorflow.keras.backend as K
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

def custom_accuracy(y_true, y_pred):
    """
    Custom accuracy metric to evaluate the model based on given conditions.
    """
    condition_1 = K.cast(y_pred < 0.5, dtype="float32") * K.cast(y_true == 0, dtype="float32")
    condition_2 = K.cast(y_pred >= 0.5, dtype="float32") * K.cast(y_true == 1, dtype="float32")
    return K.mean(condition_1 + condition_2)


def prepare_dataset(game_logs_file, features_file):
    """
    Prepare dataset for training using transformed features (difference and ratio).
    """
    try:
        # Load game logs and team features
        game_logs = pd.read_csv(game_logs_file, parse_dates=["GAME_DATE"])
        team_features = pd.read_csv(features_file)

        # Normalize key columns
        game_logs["TEAM_SEASON1"] = game_logs["TEAM_SEASON1"].str.strip().str.upper()
        game_logs["TEAM_SEASON2"] = game_logs["TEAM_SEASON2"].str.strip().str.upper()

        # Merge features for TEAM1 and TEAM2
        game_logs = game_logs.merge(
            team_features.add_suffix("_TEAM1"),
            left_on=["TEAM_SEASON1"],
            right_on=["TEAM_SEASON_TEAM1"],
            how="left"
        ).merge(
            team_features.add_suffix("_TEAM2"),
            left_on=["TEAM_SEASON2"],
            right_on=["TEAM_SEASON_TEAM2"],
            how="left"
        )

        # Drop unnecessary columns
        columns_to_drop = ["TEAM_SEASON_TEAM1", "TEAM_SEASON_TEAM2", "GAME_DATE"]
        game_logs.drop(columns=[col for col in columns_to_drop if col in game_logs.columns], inplace=True)

        # Handle missing values
        game_logs.fillna(0, inplace=True)

        # Extract features and target
        feature_columns_team1 = [col for col in game_logs.columns if col.endswith("_TEAM1")]
        feature_columns_team2 = [col.replace("_TEAM1", "_TEAM2") for col in feature_columns_team1]

        # Ensure column alignment
        feature_columns_team2 = [col for col in feature_columns_team2 if col in game_logs.columns]

        # Compute transformed features: (Team1 - Team2) / (Team1 + Team2)
        X = []
        for t1, t2 in zip(feature_columns_team1, feature_columns_team2):
            difference = game_logs[t1] - game_logs[t2]
            summation = game_logs[t1] + game_logs[t2]
            transformed_feature = difference / (summation + 1e-5)  # Add small value to avoid division by zero
            X.append(transformed_feature)

        X = np.stack(X, axis=-1)
        y = (game_logs["WL"] == "W").astype(int).to_numpy()

        return X, y, feature_columns_team1

    except Exception as e:
        print("An error occurred in prepare_dataset:", e)
        raise


def build_neural_network(input_shape):
    """
    Build a neural network model with added regularization.
    """
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),  # Reduced learning rate
                  loss="binary_crossentropy",
                  metrics=[custom_accuracy])

    print(model.summary())

    return model


def train_model(X, y, save_data_prefix="train_data"):
    """
    Train a neural network model and save the processed data.
    """
    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Handle class imbalance
    X_resampled, y_resampled = SMOTE().fit_resample(X_scaled, y)

    # Save processed training data
    pd.DataFrame(X_resampled).to_csv(f"{save_data_prefix}_X.csv", index=False, header=False)
    pd.DataFrame(y_resampled).to_csv(f"{save_data_prefix}_y.csv", index=False, header=False)

    # Debugging output
    print(f"Processed training data saved to {save_data_prefix}_X.csv and {save_data_prefix}_y.csv.")
    print(f"Resampled Features Shape: {X_resampled.shape}, Resampled Target Shape: {y_resampled.shape}")

    # Build the model
    model = build_neural_network(X_resampled.shape[1])

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor="val_loss", patience=16, restore_best_weights=True)

    # Train the model
    model.fit(X_resampled, y_resampled,
              epochs=64,
              batch_size=1,
              validation_split=0.2,
              callbacks=[early_stopping])

    return model, scaler



def main():
    game_logs_file = "nba_game_logs.csv"
    features_file = "nba_team_aggregated_data.csv"

    # Prepare the dataset
    X, y, feature_columns = prepare_dataset(game_logs_file, features_file)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Validate split
    assert X_train.shape[0] == y_train.shape[0], "Mismatch in training data sizes."
    assert X_test.shape[0] == y_test.shape[0], "Mismatch in test data sizes."

    # Train neural network
    model, scaler = train_model(X_train, y_train)

    # Save the model and scaler
    model.save("model.keras")
    joblib.dump(scaler, "scaler.pkl")

    # Evaluate the model
    X_test_scaled = scaler.transform(X_test)
    test_loss, test_custom_accuracy = model.evaluate(X_test_scaled, y_test)
    print(f"Test Loss: {test_loss}, Test Custom Accuracy: {test_custom_accuracy}")

    # Predictions
    y_pred = model.predict(X_test_scaled)
    pd.DataFrame({"Actual": y_test.flatten(), "Predicted": y_pred.flatten()}).to_csv("predictions.csv", index=False)
    print("Predictions saved to predictions.csv")


if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd
import numpy as np
import joblib
from nba_api.stats.static import teams
from keras.models import load_model


def prepare_features(team_season1, team_season2, features_file, scaler, feature_columns):
    """
    Prepare input features for prediction by combining team-specific features.
    """
    try:
        # Load team features
        team_features = pd.read_csv(features_file)

        # Normalize team names
        team_features["TEAM_SEASON"] = team_features["TEAM_SEASON"].str.strip().str.upper()
        team_season1 = team_season1.strip().upper()
        team_season2 = team_season2.strip().upper()

        # Extract features for the two teams
        features_team1 = team_features[team_features["TEAM_SEASON"] == team_season1].add_suffix("_TEAM1")
        features_team2 = team_features[team_features["TEAM_SEASON"] == team_season2].add_suffix("_TEAM2")

        if features_team1.empty or features_team2.empty:
            raise ValueError(f"Features for {team_season1} or {team_season2} not found in the file.")

        # Combine features
        combined_features = pd.concat([features_team1.reset_index(drop=True),
                                        features_team2.reset_index(drop=True)], axis=1)

        # Align with feature_columns and fill missing values with 0
        combined_features = combined_features.reindex(columns=feature_columns, fill_value=0)

        # Scale features
        X = combined_features.to_numpy()
        X_scaled = scaler.transform(X)

        return X_scaled

    except Exception as e:
        print("An error occurred in prepare_features:", e)
        raise


def predict(team_season1, team_season2, model_path, scaler_path, features_file, feature_columns):
    """
    Predict the probability of Team 1 beating Team 2.
    """
    try:
        # Load the model and scaler
        model = load_model(model_path, custom_objects={"custom_accuracy": custom_accuracy})
        scaler = joblib.load(scaler_path)

        # Prepare the input features
        X_scaled = prepare_features(team_season1, team_season2, features_file, scaler, feature_columns)

        # Make predictions
        probability = model.predict(X_scaled).flatten()[0]

        print(f"Probability of {team_season1} beating {team_season2}: {probability:.2%}")
        return probability

    except Exception as e:
        print("An error occurred in predict:", e)
        raise


def custom_accuracy(y_true, y_pred):
    """
    Custom accuracy metric to evaluate the model based on given conditions.
    """
    import tensorflow.keras.backend as K
    condition_1 = K.cast(y_pred < 0.5, dtype="float32") * K.cast(y_true == 0, dtype="float32")
    condition_2 = K.cast(y_pred >= 0.5, dtype="float32") * K.cast(y_true == 1, dtype="float32")
    return K.mean(condition_1 + condition_2)


def display_team_data():
    """
    Display available team abbreviations and names for user reference.
    """
    nba_teams = teams.get_teams()
    print("Available NBA Teams:")
    for team in nba_teams:
        print(f"{team['abbreviation']} - {team['full_name']}")


if __name__ == "__main__":
    display_team_data()
    # Example usage
    team_season1 = input("Enter TEAM_SEASON like team:season (e.g., GSW:2024): ")
    team_season2 = input("Enter TEAM_SEASON like team:season (e.g., PHI:2024): ")
    model_path = "model.keras"
    scaler_path = "scaler.pkl"
    features_file = "nba_team_aggregated_data.csv"

    # Load feature columns from the training process
    feature_columns = pd.read_csv("train_data_X.csv", nrows=0).columns.tolist()

    predict(team_season1, team_season2, model_path, scaler_path, features_file, feature_columns)
